In [3]:
from gurobipy import Model, GRB, LinExpr, QuadExpr
import pandas as pd
import gurobipy
import numpy as np
import numpy.matlib
import os.path


In [4]:
import os
import sys

In [5]:
df = pd.read_excel('~/Dropbox/MIP_regr/code/empirics_mip4.xlsx', index_col=0) 

In [6]:
df

""
rate
1.99132
0.00403
2.27000
0.84833
1.89112
...
3.39706
8.93951
4.20825


In [7]:
df = df.reset_index()

In [8]:
df

,rate
0,1.99132
1,0.00403
2,2.27000
3,0.84833
4,1.89112
...,...
98,3.39706
99,8.93951
100,4.20825
101,3.43461


In [9]:
df2 = df#[['inf','inffut','inflag','lbs']]
df2.columns = ['y']


#df1 = df2.head(198)
df1 = df2
T = len(df1)

In [10]:
T

103

In [11]:
df1['X1'] = 1

In [12]:
df1

,y,X1
0,1.99132,1
1,0.00403,1
2,2.27000,1
3,0.84833,1
4,1.89112,1
...,...,...
98,3.39706,1
99,8.93951,1
100,4.20825,1
101,3.43461,1


In [13]:





Lambda_Max = 'N/A'
steps = 'N/A'
sti, sta = None, None
#LS = np.logspace(np.log10(0.01*Lambda_Max),np.log10(Lambda_Max),10)
for lmbd_ind, lmbd_param in enumerate([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]):
    model_mip = Model('Regr3')
    model_mip.Params.TIME_LIMIT = 600.0
    #model_mip.Params.Threads = 1
    model_mip.Params.LogToConsole = 0
    model_mip.Params.LogFile = 'gur_logs_application3/log_appl_'+str(lmbd_ind)+'.txt'
    print('start with br %s'%lmbd_param)
    #print(sti, sta)
    a = {}
    z = {}
    y = {}
    d = {}
    M = 20 #20

    int_mark = {}

    for i in range(T-1):
        int_mark[i] = model_mip.addVar(vtype=GRB.BINARY)

    for j in range(T):
        #y[j] = model_mip.addVar(vtype=GRB.BINARY)

        for i in range(1):
            a[j,i] = model_mip.addVar(vtype=GRB.CONTINUOUS, lb=-10, ub=10)


    for i in range(T-1):
        for j in range(1):
            model_mip.addConstr(a[i+1,j] - a[i,j] <=  M*int_mark[i])
            model_mip.addConstr(a[i+1,j] - a[i,j] >= -M*int_mark[i])

            #model_mip.addConstr(a[i+1,j] - a[i,j] >=  l*(int_mark[i]) - d[i,j]*40)
            #model_mip.addConstr(a[i+1,j] - a[i,j] <=  -l*(int_mark[i]) + (1-d[i,j])*40 )   

    # less N
    model_mip.addConstr(sum([int_mark[i] for i in int_mark ]) == lmbd_param)


    for i in range(T-2):
        model_mip.addConstr(int_mark[i] + int_mark[i+1] <= 1)

    expr_obj = QuadExpr(0)
    #expr_obj = LinExpr(0)
    for i,j in df1.head(T).iterrows():
        expr_obj.add((j['y'] - a[i,0]*j['X1']) * (j['y'] - a[i,0]*j['X1']))
        #expr_obj.add((j['y'] - a[i,0]*j['X1'] - a[i,1]*j['X2'] - a[i,2]*j['X3'] - a[i,3]))

    #expr_obj.add( lmbd_param* sum([int_mark[i] for i in int_mark ] ))  #removed 13 June



    model_mip.update()
    model_mip.setObjective(expr_obj, GRB.MINIMIZE)
    model_mip.optimize()

    A = [a[i].X for i in a if i[1] == 0]
    #B = [a[i].X for i in a if i[1] == 1]
    #C = [a[i].X for i in a if i[1] == 2]
    #D = [a[i].X for i in a if i[1] == 3]
    #B = [a[i].X for i in a if i[1] == 1]
    I = [-100]+[int_mark[i].X for i in int_mark]


    curr_res = pd.DataFrame()
    curr_res['a1'] = A
    #curr_res['a2'] = B
    #curr_res['a3'] = C
    #curr_res['a4'] = D
    curr_res['int_mark'] = I
    
    cfn = 'output_gurobi_app3/res_app_'+str(lmbd_ind)
    curr_res.to_csv(cfn+'.csv')
    fres = open(cfn+'.txt','w')
    fres.write("%s"%model_mip.status)
    fres.write("\n")
    fres.write("%s"%model_mip.ObjVal)
    fres.write("\n")
    fres.write("%s"%model_mip.Runtime)
    fres.write("\n")
    fres.write("%s"%lmbd_param)
    fres.write("\n")
    fres.write("%s"%Lambda_Max)
    fres.write("\n")
    fres.write("%s\n"%steps)
    fres.write("%s\n"%model_mip.MIPGap)
    fres.close() 
    print("%s complete"%lmbd_ind)



Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-29
Set parameter TimeLimit to value 600
start with br 0
0 complete
Set parameter TimeLimit to value 600
start with br 1
1 complete
Set parameter TimeLimit to value 600
start with br 2
2 complete
Set parameter TimeLimit to value 600
start with br 3
3 complete
Set parameter TimeLimit to value 600
start with br 4
4 complete
Set parameter TimeLimit to value 600
start with br 5
5 complete
Set parameter TimeLimit to value 600
start with br 6
6 complete
Set parameter TimeLimit to value 600
start with br 7
7 complete
Set parameter TimeLimit to value 600
start with br 8
8 complete
Set parameter TimeLimit to value 600
start with br 9
9 complete
Set parameter TimeLimit to value 600
start with br 10
10 complete


In [ ]:
n_br += [len(np.where(df.int_mark > 0.5)[0])]
br += [np.where(df.int_mark > 0.5)[0]]
at = df1['y_%s'%col].to_numpy()-df1['x_%s'%col].to_numpy()*df['a1'].to_numpy()
at = np.sum(at**2)
at /= length
at = np.log(at) + 1/np.sqrt(length)*(len(np.where(df.int_mark > 0.5)[0])+1)

IC += [at]

am = np.argmin(IC)

In [14]:
print('i MSE CR GAP')
for i in range(0, 10):
    df=pd.read_csv(f'/Users/username/output_gurobi_app3/res_app_{i}.csv')
    length1 = len(df)
    #n_br += [len(np.where(df.int_mark > 0.5)[0])]
    #br += [np.where(df.int_mark > 0.5)[0]]
    at = df1['y'].to_numpy()-df1['X1'].to_numpy()*df['a1'].to_numpy()
    at = np.sum(at**2)
    at1 = at
    at /= length1
    at = np.log(at) + 1/np.sqrt(length1)*1*(len(np.where(df.int_mark > 0.5)[0])+1)
    f = open(f'/Users/username/output_gurobi_app3/res_app_{i}.txt')
    d = f.readlines()
    f.close()
    print("%s %.4f %.4f %.4f"%(i, at1, at, float(d[-1])))

i MSE CR GAP
0 1214.9219 2.5662 0.0000
1 644.9955 2.0316 0.0000
2 455.9502 1.7833 0.0000
3 406.7427 1.7676 0.0000
4 353.8350 1.7268 0.0000
5 333.0634 1.7648 0.0000
6 303.8467 1.7715 0.0000
7 288.9706 1.8199 0.0000
8 275.5868 1.8710 0.0000
9 260.7108 1.9140 0.0000


In [15]:
print('i MSE CR GAP')
for i in range(0, 10):
    df=pd.read_csv(f'/Users/username/output_gurobi_app3/res_app_{i}.csv')
    length1 = len(df)
    br = [np.where(df.int_mark > 0.5)[0]]
    print(i, br)

i MSE CR GAP
0 [array([], dtype=int64)]
1 [array([79])]
2 [array([47, 79])]
3 [array([47, 76, 82])]
4 [array([47, 76, 82, 88])]
5 [array([47, 71, 76, 82, 88])]
6 [array([47, 55, 71, 76, 82, 88])]
7 [array([47, 55, 71, 76, 82, 84, 88])]
8 [array([47, 55, 63, 65, 67, 76, 82, 88])]
9 [array([47, 55, 63, 65, 67, 76, 82, 84, 88])]
